In [77]:
%matplotlib inline
import os
import numpy as np # linear algebra
import pandas as pd # data processing
import seaborn as sns # visualizations
from PIL import Image # image processing
import matplotlib.pyplot as plt # visualizations

#PyTorch
import torch
import torch.nn as nn 
from torch.autograd import Variable
import torchvision
from torchvision import datasets, models,transforms,datasets
import torch.nn as nn
import torch.nn.functional as F
import random, time
from sklearn import metrics
import itertools
import torch.optim as optim
from torch.optim import lr_scheduler

print(torch.__version__)  #version de Pytorch

1.8.1+cu101


In [81]:
diseases = ['Tomato___Late_blight', 'Tomato___healthy', 'Grape___healthy', 'Orange___Haunglongbing_(Citrus_greening)', 'Soybean___healthy', 'Squash___Powdery_mildew', 'Potato___healthy', 'Corn_(maize)___Northern_Leaf_Blight', 'Tomato___Early_blight', 'Tomato___Septoria_leaf_spot', 'Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot', 'Strawberry___Leaf_scorch', 'Peach___healthy', 'Apple___Apple_scab', 'Tomato___Tomato_Yellow_Leaf_Curl_Virus', 'Tomato___Bacterial_spot', 'Apple___Black_rot', 'Blueberry___healthy', 'Cherry_(including_sour)___Powdery_mildew', 'Peach___Bacterial_spot', 'Apple___Cedar_apple_rust', 'Tomato___Target_Spot', 'Pepper,_bell___healthy', 'Grape___Leaf_blight_(Isariopsis_Leaf_Spot)', 'Potato___Late_blight', 'Tomato___Tomato_mosaic_virus', 'Strawberry___healthy', 'Apple___healthy', 'Grape___Black_rot', 'Potato___Early_blight', 'Cherry_(including_sour)___healthy', 'Corn_(maize)___Common_rust_', 'Grape___Esca_(Black_Measles)', 'Raspberry___healthy', 'Tomato___Leaf_Mold', 'Tomato___Spider_mites Two-spotted_spider_mite', 'Pepper,_bell___Bacterial_spot', 'Corn_(maize)___healthy']
print(len(diseases))

38


In [79]:
class Model(nn.Module):

    def __init__(self):
        super(Model, self).__init__()
        
        self.categories = 38
        
        self.block1 = nn.Sequential(  
            nn.Conv2d(in_channels = 3, out_channels = 128, kernel_size = 3, padding = 1),
            nn.LeakyReLU(inplace = True),
            nn.BatchNorm2d(128),
            nn.MaxPool2d(kernel_size = 2)) 
        
        self.block2 = nn.Sequential(          
            nn.Conv2d(in_channels = 128, out_channels = 256, kernel_size = 3, padding = 1),
            nn.LeakyReLU(inplace = True),
            nn.BatchNorm2d(256),
            nn.MaxPool2d(kernel_size = 2))
            
        self.block3 = nn.Sequential(              
            nn.Conv2d(in_channels = 256, out_channels = 512, kernel_size = 3, padding = 1),
            nn.LeakyReLU(inplace = True),
            nn.BatchNorm2d(512),
            nn.MaxPool2d(kernel_size = 2))


        
        self.classify = nn.Sequential(

            nn.Dropout(p=0.2),
            nn.Linear(in_features = 512 * 8 * 8, out_features = 512),
            nn.LeakyReLU(inplace = True),

            nn.Linear(in_features = 512, out_features = self.categories),
        )       

    def forward(self, x):
        x1 = self.block1(x)
        x2 = self.block2(x1)
        x3 = self.block3(x2)
        x4 = x3.view(x3.size(0), -1)
        x5 = self.classify(x4)
        return x5, x4, x3, x2 , x1, x

# instantiate model
Model = Model()
Model.load_state_dict(torch.load('/content/drive/MyDrive/Model_CNN'))
Model.eval()

Model(
  (block1): Sequential(
    (0): Conv2d(3, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): LeakyReLU(negative_slope=0.01, inplace=True)
    (2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (block2): Sequential(
    (0): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): LeakyReLU(negative_slope=0.01, inplace=True)
    (2): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (block3): Sequential(
    (0): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): LeakyReLU(negative_slope=0.01, inplace=True)
    (2): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)


In [94]:
import cv2
img = torch.tensor(cv2.imread('/content/026bd735-b9f4-4eab-86f3-23df15dbec95___R.S_HL 7938 copy.jpg'))
img = img.reshape(1,3,256,256)
img = img.float()
img = img.cuda()
diseases[model(img).argmax().item()]

'Tomato___healthy'